In [1]:
#We are going to use mne to import tables
import os
from tqdm import tqdm #Library used to see the progress of our loop
import mne
import pandas as pd
import numpy as np

There is some information about the data in a file called `French_information_.xlsx` . We get that information first:

In [2]:
df_info = pd.read_excel("SEED_FRA/French_information_.xlsx")

In [3]:
n_trials = 20

trial_num = tuple(f"Trial_{i}" for i in range(n_trials+1))
start_second = df_info["Trial Start second"][:n_trials+1]
end_second = df_info["Trial End second"][:n_trials+1]

We create folders to use to divide our data for emotion

In [12]:
os.makedirs("SEED_FRA_OUTPUT/POSITIVE")
os.makedirs("SEED_FRA_OUTPUT/NEGATIVE")
os.makedirs("SEED_FRA_OUTPUT/NEUTRAL")

In [4]:
emotions_dict = {1: "POSITIVE", -1:"NEGATIVE", 0:"NEUTRAL" }
trial_emotion = [emotions_dict[x] for x in df_info[:21]["label"] ]
trial_emotion

['POSITIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'POSITIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'POSITIVE',
 'NEGATIVE',
 'NEUTRAL',
 'POSITIVE',
 'POSITIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'POSITIVE',
 'NEGATIVE',
 'NEUTRAL',
 'POSITIVE']

See which channels are used and see if the order is correct

In [10]:
#Reading a cnt
def read_cnt(_path, _name, _trial_num, _start_second, _end_second):
    """Reads a .cnt file and stores in a parquet file the value of each signal"""
    eeg_raw = mne.io.read_raw_cnt(_path, preload=True)
    useless_ch = ['M1', 'M2', 'VEO', 'HEO']
    eeg_raw.drop_channels(useless_ch)
    new_ch = eeg_raw.ch_names
    
    # Aplicar filtro pasa-banda para eliminar ruido
    l_freq = 0.5  # Frecuencia de corte baja
    h_freq = 55  # Frecuencia de corte alta
    eeg_raw.filter(l_freq, h_freq)
    
    sample_freq = 1000
    
    for trial, start, end, i in tqdm(zip(_trial_num, _start_second, _end_second, range(n_trials+1)), desc="Processing", total=len(_trial_num)):
        
        #Para comprobar que todas las frecuencias de muestreo
        #son las mismas
        ####
        print(eeg_raw.info['sfreq'])
        ####
        
        start_sample = int(start * eeg_raw.info['sfreq'] )
        end_sample = int(end * eeg_raw.info['sfreq'] )
        #start_sample = int(start * sample_freq)
        #end_sample = int(end * sample_freq)

        # Read data in chunks and concatenate them
#         chunk_size = 10000
#         chunks = []
#         for chunk_start in range(start_sample, end_sample, chunk_size):
#             chunk_end = min(chunk_start + chunk_size, end_sample)
#             chunk_data = eeg_raw.get_data(start=chunk_start, stop=chunk_end)
#             chunks.append(chunk_data)
        eeg_trial = eeg_raw.copy().crop(tmin=start_sample / eeg_raw.info['sfreq'], tmax=end_sample / eeg_raw.info['sfreq'])
        
        
        #data_name = np.concatenate(chunks, axis=1).T
        #df = pd.DataFrame(data_name, columns=new_ch)
        new_file_name = f"{_name.rstrip('.cnt')}_trial_{i}_signals.fif"
        output_path = f"SEED_FRA_OUTPUT/{trial_emotion[i]}/{new_file_name}"
        #df.to_parquet(f"SEED_FRA_OUTPUT/{trial_emotion[i]}/{new_file_name}", compression='gzip')
        eeg_trial.save(output_path, overwrite=True)

In [6]:
#Gets the file names in the folder and selects just those with .cnt extension
file_names = os.listdir("SEED_FRA/EEG_raw")
cnt_names = tuple(filter(lambda s: ".cnt" in s, file_names))

In [13]:
for file_name in cnt_names:
    read_cnt(f"SEED_FRA/EEG_raw/{file_name}", file_name, trial_num, start_second, end_second)

Reading 0 ... 3815559  =      0.000 ...  3815.559 secs...


C:\Users\marco\AppData\Local\Temp\ipykernel_14260\1741641518.py:4: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  eeg_raw = mne.io.read_raw_cnt(_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 55 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 55.00 Hz
- Upper transition bandwidth: 13.75 Hz (-6 dB cutoff frequency: 61.88 Hz)
- Filter length: 6601 samples (6.601 sec)



Processing:   0%|                                                                               | 0/21 [00:00<?, ?it/s]

1000.0
Writing C:\Users\marco\Documents\MASTER_DATA_SCIENTIST\TFM_MSD\SEED_FRA_OUTPUT\POSITIVE\1_1_trial_0_signals.fif


...

Processing:  90%|███████████████████████████████████████████████████████████████▎      | 19/21 [00:37<00:03,  1.95s/it]

1000.0
Writing C:\Users\marco\Documents\MASTER_DATA_SCIENTIST\TFM_MSD\SEED_FRA_OUTPUT\NEUTRAL\8_3_trial_19_signals.fif
Closing C:\Users\marco\Documents\MASTER_DATA_SCIENTIST\TFM_MSD\SEED_FRA_OUTPUT\NEUTRAL\8_3_trial_19_signals.fif
[done]


C:\Users\marco\AppData\Local\Temp\ipykernel_14260\1741641518.py:44: RuntimeWarning: This filename (C:\Users\marco\Documents\MASTER_DATA_SCIENTIST\TFM_MSD\SEED_FRA_OUTPUT\NEUTRAL\8_3_trial_19_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_trial.save(output_path, overwrite=True)
Processing:  95%|██████████████████████████████████████████████████████████████████▋   | 20/21 [00:38<00:01,  1.64s/it]

1000.0
Writing C:\Users\marco\Documents\MASTER_DATA_SCIENTIST\TFM_MSD\SEED_FRA_OUTPUT\POSITIVE\8_3_trial_20_signals.fif
Closing C:\Users\marco\Documents\MASTER_DATA_SCIENTIST\TFM_MSD\SEED_FRA_OUTPUT\POSITIVE\8_3_trial_20_signals.fif
[done]


C:\Users\marco\AppData\Local\Temp\ipykernel_14260\1741641518.py:44: RuntimeWarning: This filename (C:\Users\marco\Documents\MASTER_DATA_SCIENTIST\TFM_MSD\SEED_FRA_OUTPUT\POSITIVE\8_3_trial_20_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_trial.save(output_path, overwrite=True)
Processing: 100%|██████████████████████████████████████████████████████████████████████| 21/21 [00:39<00:00,  1.88s/it]
